In [7]:
#pip install tensorflow

In [8]:
import tensorflow as tf
import numpy as np

## Red convolucional 1D: puede ser texto o puede ser un audio

In [23]:
# X: señal de entrada
# w: kernel o filtro
# p: padding (relleno con ceros)
# s: stride (paso)

def conv1d(X, w, p=0, s=1):
    X_padded = np.array(X)
    w = np.array(w)
    
    if p > 0:
        zero_pad = np.zeros(shape = p)
        # Concatenar los ceros antes y después de X_padded
        X_padded = np.concatenate([zero_pad, X_padded, zero_pad])
    
    res = []
    
    # Iterar sobre la señal con el stride especificado
    for i in range(0, int(len(X_padded) - len(w) + 1), s):
        res.append(np.sum(X_padded[i:i+w.shape[0]] * w))
        
    return res

In [25]:
# Kernel (filtro) w, es un filtro de suavizado que:
# - Reduce el ruido
# - Suaviza transiciones bruscas
# - Cada valor de salida será el promedio de tres valores de entrada
X = [0,0,0,0,0,9,9,9,9,9,0,0,0,0,0]
w = [1/3, 1/3, 1/3]

In [27]:
print("Conv1D: ", conv1d(X, w, p=0, s=1))

Conv1D:  [0.0, 0.0, 0.0, 3.0, 6.0, 9.0, 9.0, 9.0, 6.0, 3.0, 0.0, 0.0, 0.0]


In [29]:
X = [0,0,0,0,0,9,9,9,9,9,0,0,0,0,0]
w = [-1, 2, -1] #detecta cambios bruscos en la señal ya que hay un número positivo en el centro

In [31]:
print("Conv1D: ", conv1d(X, w, p=0, s=1))

Conv1D:  [0, 0, 0, -9, 9, 0, 0, 0, 9, -9, 0, 0, 0]


RESUMEN:

Valores = 0 significan "no hay cambio"

Valores > 0 significan "subida brusca"

Valores < 0 significan "bajada brusca"

## Red convolucional 2D: imágenes

In [46]:
def conv2d(X, w, p=(0,0), s=(1,1)):
    X = np.array(X)
    w = np.array(w)
    
    # Calcular dimensiones para el padding
    n1 = X.shape[0] + 2*p[0] # Altura con padding
    n2 = X.shape[1] + 2*p[1] # Anchura con padding
    
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X.shape[0], p[1]:p[1]+X.shape[1]] = X
    
    res = []
    
    # Iterar sobre la altura
    for i in range(0, int(X_padded.shape[0] - w.shape[0]+1), s[0]):
        res.append([])
        # Iterar sobre el ancho
        for j in range(0, int(X_padded.shape[1] - w.shape[1]+1), s[1]):
            # Extraer la porción de imagen del tamaño del kernel
            part_image = X_padded[i:i+w.shape[0], j:j+w.shape[1]]
            # Multiplicar elemento a elemento y sumar (convolución)
            res[-1].append(np.sum(part_image * w))
    
    return np.array(res)

In [50]:
# Imagen 9x9
# Rectángulo de nueves (5x3) en el centro
X = [[0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0]]

# Este kernel de promedio (1/9) suavizará la imagen, promediando cada pixel con sus 8 vecinos.
w = [[1/9,1/9,1/9],
     [1/9,1/9,1/9],
     [1/9,1/9,1/9]]

print("Conv2D: \n", conv2d(X, w, p=(1,1), s=(1,1)))

Conv2D: 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 3. 2. 1. 0. 0.]
 [0. 0. 2. 4. 6. 4. 2. 0. 0.]
 [0. 0. 3. 6. 9. 6. 3. 0. 0.]
 [0. 0. 3. 6. 9. 6. 3. 0. 0.]
 [0. 0. 3. 6. 9. 6. 3. 0. 0.]
 [0. 0. 2. 4. 6. 4. 2. 0. 0.]
 [0. 0. 1. 2. 3. 2. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [54]:
# Imagen 9x9
# Rectángulo de nueves (5x3) en el centro
X = [[0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,9,9,9,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0]]


w = [[-1,-1,-1],
     [-1, 8,-1],
     [-1,-1,-1]]

print("Conv2D: \n", conv2d(X, w, p=(1,1), s=(1,1)))

Conv2D: 
 [[  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  -9. -18. -27. -18.  -9.   0.   0.]
 [  0.   0. -18.  45.  27.  45. -18.   0.   0.]
 [  0.   0. -27.  27.   0.  27. -27.   0.   0.]
 [  0.   0. -27.  27.   0.  27. -27.   0.   0.]
 [  0.   0. -27.  27.   0.  27. -27.   0.   0.]
 [  0.   0. -18.  45.  27.  45. -18.   0.   0.]
 [  0.   0.  -9. -18. -27. -18.  -9.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]]


In [56]:
# Detectará:
# Bordes y esquinas (cambios bruscos)
# Valores altos en los bordes
# Valores bajos en áreas uniformes